In [1]:
%load_ext autor
%autoreload 2

In [2]:
import librosa
import torch

from MVALM.models.encoder import AudioEncoder

In [3]:
model = AudioEncoder(model_name='htsat', gradient_checkpointing=False)
print(model)

/home/t9s9/miniconda3/envs/fasttorch/lib/python3.10/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/t9s9/miniconda3/envs/fasttorch/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525541990/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


htsat - Parameters: 27,534,488; Hidden Size: 768


In [4]:
audio, sr = librosa.load('/home/t9s9/Datasets/AudioCaps/audio/val/rqfQRErjfk8_000170_000180.flac',
                         mono=True, sr=32000, duration=5)
audio_input = torch.from_numpy(audio).unsqueeze(0)
audio_input.shape

torch.Size([1, 160000])

In [5]:
with torch.no_grad():
    with torch.cuda.amp.autocast(enabled=False):
        spec = model.spectrogram(audio_input)
        print(spec.shape, spec.ndim)
    print(model.model.reshape_wav2img(spec).shape)

torch.Size([1, 1, 501, 64]) 4
torch.Size([1, 1, 256, 256])


In [11]:
with torch.no_grad():
    out = model(spec)
    print(out.shape)

self.dim=96 self.window_size=(8, 8) self.num_heads=4
torch.Size([64, 4, 64, 24])
None
----------
BIAS torch.Size([4, 64, 64])
self.dim=96 self.window_size=(8, 8) self.num_heads=4
torch.Size([64, 4, 64, 24])
torch.Size([64, 64, 64])
----------
BIAS torch.Size([4, 64, 64])
self.dim=192 self.window_size=(8, 8) self.num_heads=8
torch.Size([16, 8, 64, 24])
None
----------
BIAS torch.Size([8, 64, 64])
self.dim=192 self.window_size=(8, 8) self.num_heads=8
torch.Size([16, 8, 64, 24])
torch.Size([16, 64, 64])
----------
BIAS torch.Size([8, 64, 64])
self.dim=384 self.window_size=(8, 8) self.num_heads=16
torch.Size([4, 16, 64, 24])
None
----------
BIAS torch.Size([16, 64, 64])
self.dim=384 self.window_size=(8, 8) self.num_heads=16
torch.Size([4, 16, 64, 24])
torch.Size([4, 64, 64])
----------
BIAS torch.Size([16, 64, 64])
self.dim=384 self.window_size=(8, 8) self.num_heads=16
torch.Size([4, 16, 64, 24])
None
----------
BIAS torch.Size([16, 64, 64])
self.dim=384 self.window_size=(8, 8) self.num_he

# Test AudioSet

In [7]:
from torch.utils.data import DataLoader
from torchmetrics import MetricCollection
from torchmetrics.classification import MultilabelAveragePrecision, MultilabelPrecision, \
    MultilabelRecall, MultilabelExactMatch
from tqdm import tqdm
from MVALM.datasets.utils import multi_one_hot
from MVALM.datasets.AudioSet.audioset import AudioSetAudioOnly

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_model_htsat()
model.to(device).eval()

dataset = AudioSetAudioOnly(split='val', datasets_root='/media/t9s9/SSD_ubuntu/datasets/',
                            verbose=True, sample_rate=32000, length=32000 * 10, mono=True)
dataset.verify_files()
print(dataset)

2023-03-21 12:21:39,460 - INFO - Initialized dataset AudioSet
2023-03-21 12:21:39,487 - INFO - Loaded annotation with 17092 samples.
2023-03-21 12:21:39,494 - INFO - Verifying files for AudioSet...
2023-03-21 12:21:39,628 - WARNING - Found 6 missing files.
2023-03-21 12:21:39,628 - WARNING - Removing missing files from the dataset.


Dataset: AudioSet, Split: val, Size: 17086, Label: caption


In [9]:
def collate_fn(batch):
    audio = torch.stack([item.audio for item in batch])
    target = torch.stack([torch.from_numpy(multi_one_hot(item.label, 527)) for item in batch])
    return audio, target


dl = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True, collate_fn=collate_fn)

In [10]:
metric_kwags = dict(num_labels=dataset.num_classes)
metric = MetricCollection(
    [MultilabelAveragePrecision(**metric_kwags),
     MultilabelExactMatch(**metric_kwags),
     MultilabelPrecision(**metric_kwags),
     MultilabelRecall(**metric_kwags)]
).to(device)

In [12]:
with torch.no_grad():
    bar = tqdm(enumerate(dl), total=len(dl))
    for i, batch in bar:
        audio, label = batch
        audio = audio.to(device)
        label = label.to(device)

        logits = model(audio)  #['clipwise_output']

        step_metric = metric(logits, label.clone())
        bar.set_postfix(step_metric)

        if i == 100:
            break

for k, v in metric.compute().items():
    print(f"{k:<30}: {v.detach().cpu().item():.3f}")

  0%|          | 0/267 [00:00<?, ?it/s]/home/t9s9/miniconda3/envs/fasttorch/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Average precision score for one or more classes was `nan`. Ignoring these classes in macro-average
  warnings.warn(*args, **kwargs)
 37%|███▋      | 100/267 [00:32<00:54,  3.07it/s, MultilabelAveragePrecision=tensor(0.0620, device='cuda:0'), MultilabelExactMatch=tensor(0., device='cuda:0'), MultilabelPrecision=tensor(0.0031, device='cuda:0'), MultilabelRecall=tensor(0.0876, device='cuda:0')]


MultilabelAveragePrecision    : 0.007
MultilabelExactMatch          : 0.000
MultilabelPrecision           : 0.004
MultilabelRecall              : 0.500
